In [18]:
import pandas as pd
import numpy as np
import math
import flatlib as fl
import geopy as gp
import re

At this point, I need to make columns with actual astrology data -- sun signs and all that.

So I need to brainstorm what columns I'd like to put in my dataframes, as well as how I'd like to format these columns.

My initial thought is to make the following columns:

-Planetary Signs (includes the signs of each planet as a list)

-House Signs (includes the signs of each house as a list)

-Planetary Houses (includes which planets are in which house)

-Planetary Angles (includes which planets have an important angle between them)

The only thing is that FlatLib is giving me a little trouble with anything-to-do-with houses -- the planetary signs work great, as do the planetary angles. I think it's because FlatLib uses a weird house system ('Alcibitus', or something) -- and I want it in the normal one (which is Placidus as far as I can tell).

I just had a wild thought. What if calculate the Placidus houses by hand myself? I could just code up a function which gives the houses for a particular time and location. It's just trigonometry, how bad can it be.

Ok, this link tells me what to do roughly: https://www.astrologerdsbaquila.com/n/Placidus+House+Cusps+Calculation+-+Worked+Example-411-146/.

I'm gonna go step by step.

In [19]:
df = pd.DataFrame([['Jackson', 'July 2, 1997; 16:07', 'San Ramon, California, United States']], columns = ['name', 'birthtime', 'birthplace'])
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['birthplace'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [20]:
#This df isn't exactly right, but whatevs
df.head()

,name,birthtime,birthplace,location,point
0,Jackson,"July 2, 1997; 16:07","San Ramon, California, United States","(San Ramon, Contra Costa County, California, U...","(37.7799273, -121.9780153, 0.0)"


# Step 1: Convert LST to RAMC

... i.e. convert local sidereal time to right ascension of the MC ('Medium Coeli', also called 'Midheaven').

This means transforming the 24-hour-time into decimal-hours into decimal-hours * 15, aka the RAMC.

So in my case, I would transform 16:07 into about 16.12 which transforms into 241.75, which is my RAMC. (Actually, since we're going based on UTC, I need to subtract 7 hours from my birthtime -- aka 9:07.)

Here's a method that does this:

In [21]:
def lst_to_ramc(time):
    hr = int(time.split(':')[0])
    mn = int(time.split(':')[1])
    return (hr * 60 + mn) / 4

In [74]:
lst_to_ramc('9:07')
#It works!

136.75

# Step 2: Find MC

... aka the Medium Coeli aka the Midheaven.

This involves a constant 'e' which stands for the obliquity-of-ecliptic (of Earth), which is about 23 26' 31.5633" in degrees or about 23.44210092 in decimal degrees.

The relevant formula here is the following:

MC = arctan(tan(RAMC) / cos(e)).

In my case, I have MC = arctan(tan(241.75) / cos(23.44210092)) = 63.76 degrees or 63 45' 36".

Also, since my RAMC was in between 90 and 270 degrees, I need to add 180 degrees to my resulting MC. (If my RAMC were between 0 and 90 degrees, I would leave it alone. If it were between 270 and 360 degrees, I would add 180 degrees and reverse the sign (i.e. multiply by -1).)

So my final MC is 63 45' 36" + 180 = 243 45' 36".

Here's a method that does this:

In [137]:
def mc(ramc):
    e = 23.44210092
    mc = math.degrees(math.atan(math.tan(math.radians(ramc)) / math.cos(math.radians(e))))
    #print(mc)
    if ramc < 90:
        random = 'All is good.'
    elif ramc < 270:
        mc = mc + 180
    else:
        mc = (mc + 180) * (-1)
    #print(mc)
    '''
    print(math.modf(mc))
    dv = math.modf(mc)
    d = dv[1]
    mv = math.modf(dv[0] * 60)
    m = mv[1]
    sv = math.modf(mv[0] * 60)
    s = sv[1]
    mc_lst = [d,m,s]
    print(mc_lst)
    
    #ans = '{}d {}m {}s'.format(d,m,s)
    #return ans
    return mc_lst
    '''
    return mc

In [138]:
jackson_ramc = 136.75
mc(jackson_ramc)
#This is like 10 degrees off for me... come back to this.

134.28333428251193

# Step 3: The Ascendant

Here, we need to use the values e and ramc from before -- but we also need to use a value L, which is the latitude.

The formula is kinda long, so I'm just gonna write the method and see what happens.

In [147]:
def asc(ramc, L):
    e = 23.44210092
    numer1 = math.tan(L) * math.sin(math.radians(e))
    numer2 = math.sin(math.radians(ramc)) * math.cos(math.radians(e))
    full_numer = (-1) * (numer1 + numer2)
    denom = math.cos(math.radians(ramc))
    val = math.degrees(math.atan(denom / full_numer))
    #print(val)
    
    if val < 90 and val >= 0:
        random = 'All is good'
    #elif val < 0 and val >= -90:
    else:
        val += 180
    #print(val)
    if val < mc(ramc):
        val += 180
    else:
        val = (val + 180) * (-1)
    
    return val
    

In [148]:
jackson_L = 37.7799273
jackson_ramc = 136.75
asc(jackson_ramc, jackson_L)
#Not sure if this working exactly right... test out other cases.

227.7825804890644

# Step 4: The IC and DESC

... aka the Imum Coeli and the Descendant. Here we need e (as before). We also need mc and asc respectively.

In [154]:
def ic(mc):
    e = 23.44210092
    val = math.degrees(math.asin(math.sin(math.radians(mc)) * math.sin(math.radians(e))))
    return val

def desc(asc):
    e = 23.44210092
    val = math.degrees(math.asin(math.sin(math.radians(asc)) * math.sin(math.radians(e))))
    return val

In [155]:
jackson_mc = mc(jackson_ramc)
jackson_asc = asc(jackson_ramc, jackson_L)

print(ic(jackson_mc))
print(desc(jackson_asc))

#This seems incorrect. Fix later.

16.54684777638035
-17.135185146607768


# Step 5: 11th House Cusp

We're gonna need ramc, e, and L.

In [170]:
def eleventh(ramc,L):
    e = 23.44210092
    ra = ramc + 30
    delta_is_shrunk = False
    while delta_is_shrunk == False:
        ra_new = ramc + math.degrees(math.acos((-1) * (math.sin(math.radians(ra))) * (math.tan(math.radians(e))) * (math.tan(math.radians(L)))))/3
        if ra_new - ra < 0.001:
            delta_is_shrunk = True
            break
        ra = ra_new
    return ra_new

In [171]:
jackson_L = 37.7799273
jackson_ramc = 136.75

eleventh(jackson_ramc,jackson_L)

168.06120811367305

# Step 6: Convert 11th House Cusp to Longitude

This will just require the 11th house cusp and e.

In [174]:
def convert_eleventh(eleventh):
    e = 23.44210092
    return math.degrees(math.atan(math.tan(math.radians(eleventh)) / math.cos(math.radians(e))))

In [175]:
convert_eleventh(168.06120811367305)

-12.977903345227649

# Step 7: 12th House Cusp

This will require ramc, e, and L.

In [176]:
def twelfth(ramc,L):
    pass

# INTERRUPTION

... due to time constraints, I'm gonna just do the equal-house system. It's way quicker. (I may go back and finish up the Placidus stuff later though.) Here's how the equal-house system works:

-find the ascendant

-split up the rest of the circle into 30 degree chunks

-you're done.

Let's just write the method.

In [196]:
def equal_houses(time, L):
    
    hr = int(time.split(':')[0])
    mn = int(time.split(':')[1])
    ramc = (hr * 60 + mn) / 4

    e = 23.44210092
    numer1 = math.tan(L) * math.sin(math.radians(e))
    numer2 = math.sin(math.radians(ramc)) * math.cos(math.radians(e))
    full_numer = (-1) * (numer1 + numer2)
    denom = math.cos(math.radians(ramc))
    val = math.degrees(math.atan(denom / full_numer))
    #print(val)
    
    if val <= 90 and val >= 0:
        random = 'All is good'
    #elif val < 0 and val >= -90:
    else:
        val += 180
    #print(val)
    '''
    if val < mc(ramc):
        val += 180
    else:
        val = (val + 180) * (-1)
    '''
    #calculate mc(ramc).
    if val - mc(ramc) <= 180:
        random = 'All is good'
    else:
        ???
    
    
    houses = []
    for i in range(12):
        houses.append((val + 30 * i) % 360)
    
    return houses

In [202]:
jackson_L = 37.7799273
#jackson_time = '09:07'
#jackson_time = '16:07'
jackson_time = '23:07'
equal_houses(jackson_time, jackson_L)
#It doesn't work yet.

[100.36662105991138,
 130.36662105991138,
 160.36662105991138,
 190.36662105991138,
 220.36662105991138,
 250.36662105991138,
 280.3666210599114,
 310.3666210599114,
 340.3666210599114,
 10.366621059911381,
 40.36662105991138,
 70.36662105991138]

In [200]:
tom_cruise_L = 43.0481
#tom_cruise_time = '11:06'
#tom_cruise_time = '15:06'
#tom_cruise_time = '19:06'
equal_houses(tom_cruise_time, tom_cruise_L)

[168.67244325109274,
 198.67244325109274,
 228.67244325109274,
 258.6724432510928,
 288.6724432510928,
 318.6724432510928,
 348.6724432510928,
 18.672443251092744,
 48.672443251092744,
 78.67244325109274,
 108.67244325109274,
 138.67244325109274]